<a href="https://colab.research.google.com/github/anuraagkansara/SecureNLP/blob/master/code/preProcess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!python -m spacy download en
!python -m spacy download en_core_web_md


✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python2.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python2.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [0]:
!git clone https://github.com/anuraagkansara/SecureNLP.git

fatal: destination path 'SecureNLP' already exists and is not an empty directory.


In [0]:
import os
import spacy
import numpy as np
import pandas as pd
import nltk
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
import codecs
from nltk.corpus import stopwords
from os import listdir


In [0]:
nltk.download('stopwords')
stops = set(stopwords.words("english"))
print(stops)    

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
set([u'all', u'just', u"don't", u'being', u'over', u'both', u'through', u'yourselves', u'its', u'before', u'o', u'don', u'hadn', u'herself', u'll', u'had', u'should', u'to', u'only', u'won', u'under', u'ours', u'has', u"should've", u"haven't", u'do', u'them', u'his', u'very', u"you've", u'they', u'not', u'during', u'now', u'him', u'nor', u"wasn't", u'd', u'did', u'didn', u'this', u'she', u'each', u'further', u"won't", u'where', u"mustn't", u"isn't", u'few', u'because', u"you'd", u'doing', u'some', u'hasn', u"hasn't", u'are', u'our', u'ourselves', u'out', u'what', u'for', u"needn't", u'below', u're', u'does', u"shouldn't", u'above', u'between', u'mustn', u't', u'be', u'we', u'who', u"mightn't", u"doesn't", u'were', u'here', u'shouldn', u'hers', u"aren't", u'by', u'on', u'about', u'couldn', u'of', u"wouldn't", u'against', u's', u'isn', u'or', u'own', u'into', u'yourself'

In [0]:
nlp = spacy.load('en_core_web_md')

In [0]:
def detokenize(words):
    sentence=''
    stops = set(stopwords.words("english"))
    for token in words:
      if token not in stops and token not in ",!.:<>":
					if sentence == '': 
						sentence = token
					else: 
						sentence += ' ' + token
    #sentence = " ".join(sent)
    return sentence

In [0]:
def read_sentences(path, sentence_separator):
    
    filenames = os.listdir(path)
    sentences = []
    labels = []
    for filename in filenames:
        with codecs.open(path + filename,encoding='utf-8') as file:
            lines = file.read()
        lines = lines.split(sentence_separator)[:-1]
        for line in lines:
            l = np.array([np.array(x.split(' ')) for x in line.splitlines()])
            
            try:
                detokenized = detokenize(l[:,0])
                
                if sum([tag[1] == 'O' for tag in l]) == len(l):
                    sentence="\t".join([detokenized,"0\n"])
                    #sentence.encode('utf-8')
                    sentences.append(sentence)
                    
                    labels.append(0)
                else:
                    sentence="\t".join([detokenized,"1\n"])
                    #sentence.encode('utf-8')
                    sentences.append(sentence)
                    #sentences.append(detokenized[:-1])
                    labels.append(1)
            except:
                print("Error parsing sentence: ")
                print(filename)
                print(l)

    #print(l.shape)
    return sentences, labels



In [0]:
sentences_train, labels_train = read_sentences("SecureNLP/train/tokenized/", " \n")
sentences_train[0]

Error parsing sentence: 
apt28.tokens
[array([u'CHOPSTICK', u'B-Entity'], dtype='<U9')
 array([u'uses', u'B-Action'], dtype='<U8')
 array([u'a', u'B-Entity'], dtype='<U8')
 array([u'URL-safe', u'I-Entity'], dtype='<U8')
 array([u'Base64', u'I-Entity'], dtype='<U8')
 array([u'encoding', u'I-Entity'], dtype='<U8')
 array([u',', u'O'], dtype='<U1')
 array([u'using', u'B-Modifier'], dtype='<U10')
 array([u'an', u'B-Entity'], dtype='<U8')
 array([u'alphabet', u'I-Entity'], dtype='<U8')
 array([u'that', u'I-Entity'], dtype='<U8')
 array([u'substitutes', u'I-Entity'], dtype='<U11')
 array([u"''", u'I-Entity'], dtype='<U8')
 array([u'+', u'I-Entity'], dtype='<U8')
 array([u"''", u'I-Entity'], dtype='<U8')
 array([u'and', u'I-Entity'], dtype='<U8')
 array([u"''", u'I-Entity'], dtype='<U8')
 array([u'/', u'I-Entity'], dtype='<U8')
 array([u"''", u'I-Entity'], dtype='<U8')
 array([u'for', u'I-Entity'], dtype='<U8')
 array([u"''", u'I-Entity'], dtype='<U8')
 array([u'-', u'I-Entity'], dtype='<U8')

u'We written NetTraveler HERE HERE\t0\n'

In [0]:
print(sentences_train[9000])

This algorithmic approach proven detect – stop – exponentially modern threats	0



In [0]:
file1 = codecs.open("train.txt","w","utf-8")
for sent in sentences_train:
  #sent=sent.encode('utf-16')
  file1.write(sent)
file1.close()

In [0]:
print("\t".join([sentences_train[9000][:-1], "0"]))

This algorithmic approach has been proven to detect – and stop – exponentially more modern threats	0


In [0]:
labels_train[9000]

0

In [0]:
sentences_test, labels_test = read_sentences("SecureNLP/test_1/tokenized/", "\n\n")

In [0]:
file2 = codecs.open("test.txt","w","utf-8")
for sent in sentences_test:
  #sent=sent.encode('utf-8')
  file2.write(sent)
file2.close()

In [0]:
sentences_dev, labels_dev = read_sentences("SecureNLP/dev/tokenized/", "\n \n")

In [0]:
len(sentences_dev)

1213

In [0]:
file3 = codecs.open("dev.txt","w","utf-8")
for sent in sentences_dev:
  #sent=sent.encode('utf-8')
  file3.write(sent)
file3.close()

In [0]:
def smote_read_sentences(path, sentence_separator):
    filenames = os.listdir(path)
    result = []
    for filename in filenames:
        with open(path + filename) as file:
            lines = file.read()
        lines = lines.split(sentence_separator)[:-1]
        for line in lines:
            l = []
            for x in line.splitlines():
                w = x.split(' ')
                if w[0] == "''":
                    w[0] = '"'
                w[0] = w[0].lower()
                if len(w) == 2:
                    l.append(np.array(w))
            l = np.array(l)
            result.append(l)
    return result

In [0]:
train = smote_read_sentences("SecureNLP/train/tokenized/", " \n")

In [0]:
train[0]

array([['we', 'O'],
       ['have', 'O'],
       ['written', 'O'],
       ['about', 'O'],
       ['nettraveler', 'O'],
       ['before', 'O'],
       ['here', 'O'],
       ['and', 'O'],
       ['here', 'O'],
       ['.', 'O']], dtype='|S11')

In [0]:
test=smote_read_sentences("SecureNLP/test_1/tokenized/", "\n\n")

In [0]:
dev = smote_read_sentences("SecureNLP/dev/tokenized/", "\n \n")

In [0]:
from bs4 import BeautifulSoup
from imblearn.over_sampling import SMOTE

In [0]:
PATH = "SecureNLP/train/additional_plaintext/"
filenames = os.listdir(PATH)
additional_data = []
for filename in filenames:
    # print(filename)
    with codecs.open(PATH + filename,encoding='utf-8') as file:
        lines = file.read()
    soup = BeautifulSoup(lines, 'html.parser')
    soup.get_text()
    for section in soup.find_all('section'):
        for section in section.contents:
            if not str(section).startswith("<") and str(section) != "\n":
                doc = nlp(str(section))
                for sent in doc.sents:
                    tmp = []
                    for token in sent:
                        if token.text and token.text not in ("\n", " "):
                            tmp.append(token.text.lower())
                    additional_data.append(tmp)

UnicodeEncodeError: ignored

In [0]:
embed = [list(x[:, 0]) for x in train]

In [0]:
len(embed)

9425

In [0]:
!pip install glove_python

     |████████████████████████████████| 266kB 4.2MB/s 
  Created wheel for glove-python: filename=glove_python-0.1.0-cp27-cp27mu-linux_x86_64.whl size=418452 sha256=af376ed2d5c3aa3aca1b6906956df984735a1a69e6192712febab7edf59eeb63
  Stored in directory: /root/.cache/pip/wheels/88/4b/6d/10c0d2ad32c9d9d68beec9694a6f0b6e83ab1662a90a089a4b
Successfully built glove-python


In [0]:
from glove import Corpus, Glove
corpus = Corpus() 
corpus.fit(embed, window=10)

glove = Glove(no_components=300, learning_rate=0.05)
 
glove.fit(corpus.matrix, epochs=30, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)
glove.save('glove.model')

Performing 30 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29


In [0]:
len(glove.word_vectors[glove.dictionary['malware']])
#emb = glove.word_vectors[glove.dictionary[word]]

100

In [0]:
print(glove.dictionary)

{'rebuf\xef\xbf\xbdng': 7758, 'cussed': 3854, 'internalgettcptable2': 13606, 'both': 422, 'yellow': 6384, 'four': 1110, 'prefix': 6216, 'minhang': 8455, 'aggression': 11833, 'increase': 1118, 'netcfg.dll': 12394, 'granting': 4468, 'eligible': 7651, 'electricity': 10532, '*.evt': 1952, 'unanswered': 11553, 'superficially': 8399, '14c04f88dc97aef3e9b516ef208a2bf5': 13289, 'digit': 12988, 'statically': 6092, 'co-operation': 11271, '10/msk+7': 9251, 'regional': 8222, 'kaspersky': 81, 'replaces': 11724, 'tantalizing': 9159, '4c703a8cfeded7f889872a86fb7c70cf': 14710, 'components\xcd\xbe': 5995, 'bringing': 10872, 'modularize': 2190, '.kp': 10728, 'basics': 5993, 'internally': 6496, 'scholar': 10832, 'customizable': 6361, "'sengoku": 8882, 'cozyduke': 7211, 'wednesday': 9864, 'implanting': 5585, 'persisted': 10625, 'trojan.win32.karba.e': 9760, 'd30c1661-cdaf-11d0-8a3e-00c04fc9e26e': 5512, 'zeronights': 10178, 'hostels': 10156, 'sug-': 3960, 'langrussian': 11632, 'aaa': 8845, 'expanded': 7458